In [25]:
pip install sagemaker --upgrade

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

arn:aws:iam::204222797759:role/service-role/AmazonSageMaker-ExecutionRole-20210626T194361


In [27]:
from sagemaker.huggingface import HuggingFace, HuggingFaceModel

In [28]:
hyperparameters = {}

huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    transformers_version='4.6',
    pytorch_version='1.7',
    py_version='py36',
    hyperparameters=hyperparameters,
)

In [29]:
huggingface_estimator.fit(
    {'train': 's3://hackathongroup4/augmented_commands.csv'}
)

2021-07-23 11:01:09 Starting - Starting the training job...
2021-07-23 11:01:32 Starting - Launching requested ML instancesProfilerReport-1627038068: InProgress
...
2021-07-23 11:02:02 Starting - Preparing the instances for training.........
2021-07-23 11:03:39 Downloading - Downloading input data...
2021-07-23 11:04:03 Training - Downloading the training image................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-07-23 11:06:44,277 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-07-23 11:06:44,300 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-07-23 11:06:44,307 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-07-23 11:06:44,733 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "t

In [68]:
huggingface_estimator.model_data

's3://sagemaker-ap-southeast-1-204222797759/huggingface-pytorch-training-2021-07-23-11-01-08-350/output/model.tar.gz'

In [31]:
# # create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data=huggingface_estimator.model_data,
    transformers_version='4.6',
    pytorch_version='1.7',
    py_version='py36',
	role=role, 
)

# # deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

-------------!

In [71]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'huggingface-pytorch-inference-2021-07-23-11-14-07-476'

In [75]:
predictor.predict({'inputs': "learn stuff"})

[{'label': 'LABEL_3', 'score': 0.9333232641220093}]

In [72]:
def do_prediction(query):
    MAPPER = {
        'LABEL_0': 'orbit', 
        'LABEL_2': 'enghub', 
        'LABEL_7': 'techrisk', 
        'LABEL_8': 'teutr', 
        'LABEL_6': 'mail', 
        'LABEL_5': 'kerb', 
        'LABEL_9': 'time', 
        'LABEL_3': 'help', 
        'LABEL_4': 'im', 
        'LABEL_1': 'call'
    }
    output = predictor.predict({'inputs': query})
    return MAPPER[output[0]['label']]

In [73]:
do_prediction("siu hong phone number")

'call'

In [53]:
do_prediction("send email to binitha")

'mail'

In [54]:
do_prediction("chec k security inciedents")

'techrisk'

In [55]:
do_prediction("wjere helpdesk")

'help'

In [57]:
do_prediction("what time san franc")

'time'

In [65]:
do_prediction("get tech support")

'teutr'

In [66]:
do_prediction("engineering resourcs")

'enghub'

In [67]:
do_prediction("get in touch w pengfei")

'im'